## Method

1. Frame the problem and look at the big picture.
2. Get the data.
3. Explore the data to gain insights.
4. Prepare the data to better expose the underlying data patterns to Machine      Learning algorithms.
5. Explore many different models and shortlist the best ones.
6. Fine-tune your models and combine them into a great solution.
7. Present your solution.
8. Launch, monitor, and maintain your system.

## All imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns

from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, ExtraTreesClassifier
from sklearn.svm import LinearSVC
from sklearn.datasets import make_classification
from sklearn.pipeline import make_pipeline

In [2]:
# Set the path to housing csv file
housing = pd.read_csv(r'housing.csv')

In [3]:
def display_confusion_matrix(y_test, y_pred):
    cm = confusion_matrix(y_test, y_pred)
    ConfusionMatrixDisplay(cm).plot()

#### Storing the data in the way we want it


In [18]:
housing_clean = housing[housing["median_house_value"] <= 500000]
housing_clean = housing_clean.dropna()
housing_clean

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY
...,...,...,...,...,...,...,...,...,...,...
20635,-121.09,39.48,25.0,1665.0,374.0,845.0,330.0,1.5603,78100.0,INLAND
20636,-121.21,39.49,18.0,697.0,150.0,356.0,114.0,2.5568,77100.0,INLAND
20637,-121.22,39.43,17.0,2254.0,485.0,1007.0,433.0,1.7000,92300.0,INLAND
20638,-121.32,39.43,18.0,1860.0,409.0,741.0,349.0,1.8672,84700.0,INLAND


In [19]:
# With dummy varible convert column 'ocean_proximity' into float64 values.
ocean_proximity_matrix = pd.get_dummies(pd.Series(housing["ocean_proximity"]), dtype=float)
ocean_proximity_matrix

,<1H OCEAN,INLAND,ISLAND,NEAR BAY,NEAR OCEAN
0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...
20635,0.0,1.0,0.0,0.0,0.0
20636,0.0,1.0,0.0,0.0,0.0
20637,0.0,1.0,0.0,0.0,0.0
20638,0.0,1.0,0.0,0.0,0.0


In [20]:
# Remove cloumn 'ocean_proximity' with object values and join the new matrix where 'ocean_proximity'
# has been converted to float46-values thanks to get_dummies() function in pandas.
housing_modified = housing_clean.copy()
housing_modified = housing_modified.drop('ocean_proximity', axis=1)
housing_modified_ocean = pd.concat([housing_modified, ocean_proximity_matrix], axis=1)
housing_modified_ocean.dropna(inplace=True)
housing_modified_ocean

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,<1H OCEAN,INLAND,ISLAND,NEAR BAY,NEAR OCEAN
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,0.0,0.0,0.0,1.0,0.0
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,0.0,0.0,0.0,1.0,0.0
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,0.0,0.0,0.0,1.0,0.0
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,0.0,0.0,0.0,1.0,0.0
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20635,-121.09,39.48,25.0,1665.0,374.0,845.0,330.0,1.5603,78100.0,0.0,1.0,0.0,0.0,0.0
20636,-121.21,39.49,18.0,697.0,150.0,356.0,114.0,2.5568,77100.0,0.0,1.0,0.0,0.0,0.0
20637,-121.22,39.43,17.0,2254.0,485.0,1007.0,433.0,1.7000,92300.0,0.0,1.0,0.0,0.0,0.0
20638,-121.32,39.43,18.0,1860.0,409.0,741.0,349.0,1.8672,84700.0,0.0,1.0,0.0,0.0,0.0


In [21]:
X = housing_modified_ocean.copy() 
X = X.drop(columns = ['median_house_value'])
X

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,<1H OCEAN,INLAND,ISLAND,NEAR BAY,NEAR OCEAN
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,0.0,0.0,0.0,1.0,0.0
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,0.0,0.0,0.0,1.0,0.0
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,0.0,0.0,0.0,1.0,0.0
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,0.0,0.0,0.0,1.0,0.0
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
20635,-121.09,39.48,25.0,1665.0,374.0,845.0,330.0,1.5603,0.0,1.0,0.0,0.0,0.0
20636,-121.21,39.49,18.0,697.0,150.0,356.0,114.0,2.5568,0.0,1.0,0.0,0.0,0.0
20637,-121.22,39.43,17.0,2254.0,485.0,1007.0,433.0,1.7000,0.0,1.0,0.0,0.0,0.0
20638,-121.32,39.43,18.0,1860.0,409.0,741.0,349.0,1.8672,0.0,1.0,0.0,0.0,0.0


In [22]:
y = housing_modified_ocean['median_house_value'].copy()
y.info()

<class 'pandas.core.series.Series'>
Index: 19475 entries, 0 to 20639
Series name: median_house_value
Non-Null Count  Dtype  
--------------  -----  
19475 non-null  float64
dtypes: float64(1)
memory usage: 304.3 KB


In [9]:
y.describe()

count     19475.000000
mean     192444.651913
std       97702.625782
min       14999.000000
25%      116500.000000
50%      173800.000000
75%      248100.000000
max      500000.000000
Name: median_house_value, dtype: float64

In [10]:
y

0        452600.0
1        358500.0
2        352100.0
3        341300.0
4        342200.0
           ...   
20635     78100.0
20636     77100.0
20637     92300.0
20638     84700.0
20639     89400.0
Name: median_house_value, Length: 19475, dtype: float64

In [11]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19475 entries, 0 to 20639
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           19475 non-null  float64
 1   latitude            19475 non-null  float64
 2   housing_median_age  19475 non-null  float64
 3   total_rooms         19475 non-null  float64
 4   total_bedrooms      19475 non-null  float64
 5   population          19475 non-null  float64
 6   households          19475 non-null  float64
 7   median_income       19475 non-null  float64
 8   <1H OCEAN           19475 non-null  float64
 9   INLAND              19475 non-null  float64
 10  ISLAND              19475 non-null  float64
 11  NEAR BAY            19475 non-null  float64
 12  NEAR OCEAN          19475 non-null  float64
dtypes: float64(13)
memory usage: 2.1 MB


In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [13]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# EDA
-- Describe why we do this

In [ ]:
# Overview of the data
housing

In [ ]:
# Display information about housing data.
# "We found that the data has missing values in the column 'total_bedrooms'"
# Drop null data, as it is 1%.....
housing.info()

In [ ]:
# Why??
housing.hist(bins=50, figsize=(20, 15))
print()

In [ ]:
housing_modified_ocean.describe()

In [ ]:
# Placing all the data in "df" so I can make a nice correlation plot
df = housing_modified_ocean.copy()
df["price"] = housing_modified_ocean['median_house_value']
df.drop(columns=['median_house_value'], inplace=True)

correlation_matrix = df.corr()
fig, ax = plt.subplots(figsize=(10,10))   
ax = sns.heatmap(correlation_matrix, annot=True, linewidths=.5, ax=ax)

In [ ]:
california_img=mpimg.imread('california.png')
ax = housing.plot(kind="scatter", x="longitude", y="latitude", figsize=(10,7),
                       s=housing['population']/100, label="Population",
                       c="median_house_value", cmap=plt.get_cmap("jet"),
                       colorbar=False, alpha=0.4,
                      )
plt.imshow(california_img, extent=[-124.55, -113.80, 32.45, 42.05], alpha=0.5)
plt.ylabel("Latitude", fontsize=14)
plt.xlabel("Longitude", fontsize=14)

prices = housing["median_house_value"]
tick_values = np.linspace(prices.min(), prices.max(), 11)
cbar = plt.colorbar()
cbar.set_label('Median House Value', fontsize=16)
cbar.ax.set_yticklabels(["$%dk"%(round(v/1000)) for v in tick_values], fontsize=14)


plt.legend(fontsize=16)
plt.show()

In [ ]:
# Modellen kan inte predikta över 500 000, linjen längst upp... (om man använde housing)

fig, ax = plt.subplots(figsize=(10, 7))
ax.scatter(housing_clean['median_income'], housing_clean['median_house_value'], alpha=1)
ax.set_ylabel('Median house value', labelpad=15, fontweight='bold')
ax.set_xlabel('Median income', labelpad=15, fontweight='bold')
 
correlation = np.corrcoef(housing_clean['median_income'], housing_clean['median_house_value'])
print(f' The correlation coefficient between median house value and median income is {round(correlation[1][0], 4)}')

In [ ]:
# Modellen kan inte predikta över 500 000, linjen längst upp... (om man använde housing)

fig, ax = plt.subplots(figsize=(10, 7))
ax.scatter(housing_clean['total_rooms'], housing_clean['median_house_value'], alpha=1)
ax.set_ylabel('Median house value', labelpad=15, fontweight='bold')
ax.set_xlabel('Total rooms', labelpad=15, fontweight='bold')
 
correlation = np.corrcoef(housing_clean['total_rooms'], housing_clean['median_house_value'])
print(f' The correlation coefficient between median house value and total rooms is {round(correlation[1][0], 4)}')

## Model Selection

1. Linear Regression 
2. Lasso 
3. Random Forest Regression
4. Beslutsträd
5. SVM
6. Extra Tree

In [ ]:
y_train.describe()

In [16]:
# mean from y_train.describe(), 192150.461874
# fit_intercept=False -> If set to False, no intercept will be used in calculations (i.e. data is expected to be centered).
lin_reg = LinearRegression(fit_intercept=False)

scores_lr = cross_validate(lin_reg, X_train_scaled, y_train, cv=3, scoring = 'neg_mean_squared_error')["test_score"]
print('RMSE for each iteration:', np.sqrt(-scores_lr))
print('RMSE:', np.sqrt(np.mean(-scores_lr)))

RMSE for each iteration: [202354.515254   202585.69279008 200396.98851192]
RMSE: 201781.45419534325


In [15]:
lasso = Lasso()
hyper_param_lasso = {'alpha':(0.01, 1, 2, 5, 10, 15)}
lasso_reg = GridSearchCV(lasso, hyper_param_lasso, cv = 5)
lasso_reg.fit(X_train_scaled, y_train) 

print(lasso_reg.best_params_) 

scores_lasso = cross_validate(lasso_reg, X_train_scaled, y_train, cv=3, scoring = 'neg_mean_squared_error')["test_score"]

print('RMSE for each iteration:', np.sqrt(-scores_lasso))

print('RMSE:', np.sqrt(np.mean(-scores_lasso)))

C:\Users\Amand\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.411e+12, tolerance: 1.181e+10
  model = cd_fast.enet_coordinate_descent(
C:\Users\Amand\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.423e+12, tolerance: 1.184e+10
  model = cd_fast.enet_coordinate_descent(
C:\Users\Amand\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.424e+12, toleranc

{'alpha': 15}


C:\Users\Amand\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.150e+11, tolerance: 7.840e+09
  model = cd_fast.enet_coordinate_descent(
C:\Users\Amand\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.304e+11, tolerance: 7.875e+09
  model = cd_fast.enet_coordinate_descent(
C:\Users\Amand\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.298e+11, toleranc

C:\Users\Amand\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.360e+11, tolerance: 7.857e+09
  model = cd_fast.enet_coordinate_descent(
C:\Users\Amand\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.302e+11, tolerance: 7.854e+09
  model = cd_fast.enet_coordinate_descent(
C:\Users\Amand\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.985e+11, toleranc

C:\Users\Amand\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.909e+11, tolerance: 7.970e+09
  model = cd_fast.enet_coordinate_descent(
C:\Users\Amand\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.812e+11, tolerance: 7.942e+09
  model = cd_fast.enet_coordinate_descent(
C:\Users\Amand\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.191e+10, toleranc

RMSE for each iteration: [61791.47283805 61622.68304236 59612.23125243]
RMSE: 61016.8263561466


C:\Users\Amand\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.575e+10, tolerance: 9.931e+09
  model = cd_fast.enet_coordinate_descent(


In [17]:
forest = RandomForestRegressor()
param_grid = { 
    'n_estimators': [10, 100, 120],
    'max_depth' : [None, 10],
}
forest_reg = GridSearchCV(forest, param_grid=param_grid, cv= 5)
forest_reg.fit(X_train_scaled, y_train)

print(forest_reg.best_params_)

scores_forest = cross_validate(forest_reg, X_train_scaled, y_train, cv=3, scoring = 'neg_mean_squared_error')["test_score"]
print('RMSE for each iteration:', np.sqrt(-scores_forest))
print('RMSE:', np.sqrt(np.mean(-scores_forest)))

{'max_depth': None, 'n_estimators': 120}
RMSE for each iteration: [46085.22037223 46913.85281224 45648.86972048]
RMSE: 46218.95869735533


In [ ]:
forest = RandomForestRegressor()
param_grid = { 
    'n_estimators': [10, 100, 120],
    'max_depth' : [None, 5, 10],
}
forest_reg = GridSearchCV(forest, param_grid=param_grid, cv= 5)
forest_reg.fit(X_train_scaled, y_train)

print(forest_reg.best_params_)

scores_forest = cross_validate(forest_reg, X_train_scaled, y_train, cv=3, scoring = 'neg_mean_squared_error')["test_score"]
print('RMSE for each iteration:', np.sqrt(-scores_forest))
print('RMSE:', np.sqrt(np.mean(-scores_forest)))

In [ ]:
# havn't test runned it 
svm_clf = LinearSVC(max_iter=5000, random_state=42)
svm_clf.fit(X_train_scaled, y_train)
svm_clf.score(X_test, y_test)

In [ ]:
# havn't test runned it 
clf = make_pipeline(StandardScaler(), LinearSVC(random_state=0, tol=1e-5))
clf.fit(X_train_scaled, y_train)
clf.score(X_test, y_test)